# Large Language Models (LLMs)

Large Language Models (LLMs) are a core component of LangChain. LangChain does not serve its own LLMs, but rather provides a standard interface for interacting with many different LLMs. To be specific, this interface is one that takes as input a string and returns a string.

## LLM setup

In [ ]:
import os
from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

print(prompt.format(product="podcast player"))

In [ ]:
llm = OpenAI(
          model_name="text-davinci-003",    # default model
          temperature=0.9                   #temperature dictates how whacky the output should be
)

### Azure OpenAI

Azure OpenAI Service provides REST API access to OpenAI’s powerful language models including the GPT-4, GPT-3.5-Turbo, and Embeddings model series.

In [ ]:
import os
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")
os.environ["AZURE_OPENAI_ENDPOINT"] = input("Enter your Astra DB API Endpoint: ")

In [ ]:
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4",
)

In [ ]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm([message])

### OpenAI

### Google Cloud Vetex AI

### Huggingface

To instantiate a Hugging Face LLM there are three options available

* HuggingFaceTextGenInference
* HuggingFaceEndpoint
* HuggingFaceHub

In [ ]:
import os
from langchain_community.llms import HuggingFaceTextGenInference

### Ollama

Ollama allows you to run open-source large language models, such as Google's Gemma, LlaMA 2 or Mistral locally. Ollama supports importing [models](https://ollama.com/library) in GGUF format (GPT-Generated Unified Format). Ollama bundles model weights, configuration, and data into a single package, defined by a Modelfile. It optimizes setup and configuration details, including CPU and GPU usage.

### Gemma7B

Gemma is a family of lightweight, state-of-the-art open models built by Google DeepMind. It’s inspired by Gemini models at Google.
Gemma is available in both 2b and 7b parameter sizes. The models undergo training on a diverse dataset of web documents to expose them to a wide range of linguistic styles, topics, and vocabularies. This includes code to learn syntax and patterns of programming languages, as well as mathematical text to grasp logical reasoning.
To ensure the safety of the model, the team employed various data cleaning and filtering techniques, including rigorous filtering for CSAM (child sexual abuse material), sensitive data filtering, and filtering based on content quality in compliance with Google’s policies.

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(base_url="http://192.168.178.84:11434", model="gemma:7b")
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | llm | StrOutputParser()
topic = {"topic": "Artificial Intelligence"}

print(chain.invoke(topic))

Why did the AI scientist cross the road?

To get to the other algorithm side.


In [ ]:
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import HumanMessage

llm = ChatOllama(model="gemma:7b", base_url = "http://192.168.178.84:11434", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

messages = [
    HumanMessage(content="Why is the sky blue?")
]
llm(messages)

In [10]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from IPython.display import display, Markdown
from langchain.chains import LLMChain

llm = ChatOllama(model="gemma:7b", base_url = "http://192.168.178.84:11434")

prompt_template = PromptTemplate.from_template(
    "Why is the {topic} {color}."
)

prompt = prompt_template.format(topic="car", color="red")

print(prompt)

Why is the car red.


In [15]:
chain = LLMChain(llm=llm, prompt=prompt_template)
result = chain.run(topic="sky", color="blue")
display(Markdown(result))

The sky appears blue due to a phenomenon called **Rayleigh Scattering**.

**Rayleigh Scattering:**

* Sunlight consists of various colors of the rainbow, including blue, red, green, and yellow.
* As sunlight enters the Earth's atmosphere, particles of air scatter the different colors of light in all directions.
* Scattered light, including the blue component, is scattered more effectively in the direction of the observer's eyes.

**Scattering Intensity:**

* The intensity of scattered light increases with the wavelength of light (shorter wavelengths like blue light scatter more effectively).
* The scattered light is scattered in all directions, but the light scattered in the direction of the observer's eyes is more noticeable.

**Appearance of the Sky:**

* The scattered blue light from various air particles accumulates in the atmosphere, making the sky appear blue.
* The intensity of blue color increases as the sun's rays pass through a greater amount of air, such as at sunrise and sunset.

**Other Factors:**

* The time of day and angle of the sun can influence the intensity of the blue sky.
* Clouds and pollution can sometimes reduce the scattering of blue light, making the sky appear white or gray.

**Conclusion:**

Rayleigh scattering of sunlight by particles in the atmosphere causes the sky to appear blue. The scattered light in the direction of the observer's eyes is predominantly blue, giving the sky its characteristic color.

### Mistral7B

[Mistral is a 7.3B](https://mistral.ai/news/announcing-mistral-7b/) parameter model, distributed with the Apache license. It is available in both instruct (instruction following) and text completion. The Mistral AI team has noted that Mistral 7B:

* Outperforms Llama 2 13B on all benchmarks
* Outperforms Llama 1 34B on many benchmarks
* Approaches CodeLlama 7B performance on code, while remaining good at English tasks

Mistral7B can easy deploy in the cloud (AWS/GCP/Azure) using [vLLM](https://docs.mistral.ai/self-deployment/skypilot/) or locally via Ollama. 

### Multi-modal with LlaVA

[LLaVA](https://llava-vl.github.io) is a multimodal model that combines a vision encoder and Vicuna for general-purpose visual and language understanding, achieving impressive chat capabilities mimicking spirits of the multimodal GPT-4. LLaVA 1.6 supporting 672x672, 336x1344, 1344x336 resolutions, a better visual reasoning and OCR capability with an improved visual instruction tuning data mixture.

In [3]:
import base64
from io import BytesIO
from PIL import Image
from datetime import datetime
from IPython.display import HTML, display
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [4]:
def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Re-sized Base64 string
    """

    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]

def plt_img_base64(img_base64):
    """
    Disply base64 encoded string as image

    :param img_base64:  Base64 string
    """
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))

In [5]:
file_path = "../assets/teatro-alla-scala.jpeg"
pil_image = Image.open(file_path)

image_b64 = convert_to_base64(pil_image)
plt_img_base64(image_b64)

 The image shows a building with classical architectural features, such as columns and a pediment. Above the entrance, there is a sign that reads "Teatro alla Scala," which translates to "Theater at La Scala." This suggests that the building is a theater, specifically La Scala, which is a famous opera house in Milan, Italy. The facade of the building has Italian flags on either side, indicating a connection to Italy or an event related to Italian culture. In front of the building, there are people standing and walking around, suggesting that this might be a public area where performances take place or where visitors gather before entering the theater. The sky is clear, and it appears to be late afternoon or early evening, as indicated by the soft lighting. 
Time elapsed (hh:mm:ss.ms) 0:34:22.660857


In [ ]:
llm = ChatOllama(base_url="http://192.168.178.84:11434", model="llava:v1.6", temperature=0)

chain = prompt_func | llm | StrOutputParser()

start_time = datetime.now()
query_chain = chain.invoke(
    {"text": "What is on this image?", "image": image_b64}
)

print(query_chain)

time_elapsed = datetime.now() - start_time
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

## Streaming

## Temperature

## Tracking token usage

## LangChain Expression Language (LCEL)